# 0. Load data

In [23]:

import tensorflow_hub as hub
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql import Window as w

from pyspark.ml.linalg import DenseVector, SparseVector
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer, CountVectorizer, StopWordsRemover, NGram, Normalizer, VectorAssembler, Word2Vec, Word2VecModel, PCA
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.linalg import VectorUDT, Vectors

spark = SparkSession.builder.getOrCreate()

In [24]:
df = spark.read.csv("data/X2.csv", header=True)
df.toPandas()

,instance_id,brand,cpu_brand,cpu_model,cpu_type,cpu_frequency,ram_capacity,ram_type,ram_frequency,hdd_capacity,ssd_capacity,weight,dimensions,title
0,www.softwarecity.ca//737,Lenovo,Intel. i5-3320M,i5-3320M,Dual-core ( 2 Core ). Core i5,2.60 GHz,None,DDR3 SDRAM. DDR3-1600/PC3-12800. DDR3 SDRAM,DDR3-1600/PC3-12800,320 GB,None,1.80 kg,None,"""Lenovo Thinkpad X230 34352jf Tablet Pc - 12.5..."
1,www.isupplyhub.com//1256,Acer,1.6 GHz Intel Core i5-4200U. Intel Core I5,None,1.6 GHz Intel Core i5-4200U,1.6 GHz Intel Core i5-4200U,8 GB DDR3L SDRAM,DDR3 SDRAM. 8 GB DDR3L SDRAM,None,500 GB mechanical_hard_drive,None,4.8 pounds,15.02 x 10.08 x 0.90 inches,Amazon.com : Acer Aspire V7-582PG-6479 15.6-In...
2,www.isupplyhub.com//326,Acer,1.6 GHz Intel Core i5. Intel Core I5,None,1.6 GHz Intel Core i5,1.6 GHz Intel Core i5,4 GB DDR3-SDRAM,DDR3 SDRAM. 4 GB DDR3-SDRAM,None,500 GB mechanical_hard_drive,None,5.2 pounds,15.02 x 10.08 x 1 inches,Amazon.com : Acer Aspire E1-572-6870 15.6 Inch...
3,www.isupplyhub.com//821,HP,None,None,None,None,4 GB SDRAM DDR3,DDR3 SDRAM. 4 GB SDRAM DDR3,None,500 GB,None,4.8 pounds,15.18 x 0.89 x 10.16 inches,"""Amazon.com : 15.6"""" HP 15-f009wm Amd Dual-Cor..."
4,www.isupplyhub.com//157,Asus,1.7 GHz Core i5-3317U. Intel,None,1.7 GHz Core i5-3317U,1.7 GHz Core i5-3317U,4 GB DDR3,DDR3 SDRAM. 4 GB DDR3,None,256 MB,None,2.9 pounds,8.80 x 0.70 x 12.80 inches,Amazon.com : ASUS UX31A-XB52 13.3-Inch Ultrabo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,www.vology.com//873,Lenovo ThinkPad X230 2320 - 12.5 '' - Core i5 ...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,180 GB SSD. 180 GB SSD. Lenovo ThinkPad X230 2...,180 GB SSD. 180 GB SSD,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,"""Lenovo ThinkPad X230 2320 - 12.5"""" - Core i5 ..."
339,www.vology.com//823,Lenovo ThinkPad X230 2325 - 12.5 '' - Core i5 ...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,4 GB DDR3 Slots Qty 2 Max RAM Supported 16 GB ...,4 GB DDR3 Slots Qty 2 Max RAM Supported 16 GB ...,4 GB DDR3 Slots Qty 2 Max RAM Supported 16 GB ...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm. ...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,"""Lenovo ThinkPad X230 2325 - 12.5"""" - Core i5 ..."
340,www.vology.com//2723,Lenovo ThinkPad X230 Tablet 3438 - 12.5 '' - C...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm. ...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm,4 lbs 4 lbs,9 in. 12 in x 9 in x 1.2 in. 1.2 in. 12 in,"""Lenovo ThinkPad X230 Tablet 3438 - 12.5"""" - C..."
341,www.vology.com//1349,Lenovo ThinkPad X230 2324 - 12.5 '' - Core i5 ...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,320 GB HDD / 7200 rpm. 320 GB HDD / 7200 rpm. ...,320 GB HDD / 7200 rpm. 320 GB HDD / 7200 rpm,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,"""Lenovo ThinkPad X230 2324 - 12.5"""" - Core i

# 1. Blocking

In [25]:
"""UTILITIES"""

"""Returns the df with tokenized columns with stopwords removed"""
def tokenize(df, string_cols):
  output = df
  for c in string_cols:
    output = output.withColumn('temp', f.coalesce(f.col(c), f.lit('')))
    tokenizer = RegexTokenizer(inputCol='temp', outputCol=c+"_tokens", pattern = "\\W")
    remover = StopWordsRemover(inputCol=c+"_tokens", outputCol=c+"_swRemoved")
    output = tokenizer.transform(output)
    output = remover.transform(output)\
      .drop('temp', c+"_tokens")
    # output has c+swRemoved columns
  return output

"""Selects the n top keywords from the tokens in each row from a vocab(from one col) using TFIDF score"""
def top_kw_from_tfidf(vocab, n=3):
  @f.udf(returnType=t.ArrayType(t.StringType()))
  def _(arr):
    inds = arr.indices
    vals = arr.values
    top_inds = vals.argsort()[-n:][::-1]
    top_keys = inds[top_inds]
    output = []

    for k in top_keys:
      kw = vocab.value[k]
      output.append(kw)

    return output
  return _

"""Selects top n tokens using TFIDF score for each token_cols using above func"""
def tfidf_top_tokens(df, token_cols, min_freq=1):
  output = df
  for c in token_cols:
    pre = c
    cv = CountVectorizer(inputCol=pre, outputCol=pre+'_rawFeatures', minDF=1, minTF=min_freq)
    idf = IDF(inputCol=pre+"_rawFeatures", outputCol=pre+"_features", minDocFreq=1)
    normalizer = Normalizer(p=2.0, inputCol=pre+"_features", outputCol=pre+'_tfidf')
    stages = [cv, idf, normalizer]
    pipeline = Pipeline(stages=stages)
    model = pipeline.fit(output)
    output = model.transform(output)\
      .drop(pre+'_rawFeatures', pre+'_features')

    cvModel = model.stages[0]
    vocab = spark.sparkContext.broadcast(cvModel.vocabulary)
    output = output.withColumn(pre+'_top_tokens', top_kw_from_tfidf(vocab, n=5)(f.col(pre+"_tfidf")))

  return output

"""Use universal sentence encoder from tensorflow_hub"""
MODEL = None
def get_model_magic():
  global MODEL
  if MODEL is None:
      MODEL = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
  return MODEL

@f.udf(returnType=VectorUDT())
def encode_sentence(x):
  model = get_model_magic()
  emb = model([x]).numpy()[0]
  return Vectors.dense(emb)
columns = ['title']
blocking_df = tokenize(df, columns)
#use N-Gram

In [26]:
blocking_df = tfidf_top_tokens(blocking_df, [c+'_swRemoved' for c in columns])


In [27]:
blocking_df = blocking_df.withColumn('title_encoding', encode_sentence(f.coalesce(f.col('title'), f.lit(''))))\
.withColumn('blocking_keys',f.col('title_swRemoved_top_tokens'))
blocking_df.toPandas()

,instance_id,brand,cpu_brand,cpu_model,cpu_type,cpu_frequency,ram_capacity,ram_type,ram_frequency,hdd_capacity,ssd_capacity,weight,dimensions,title,title_swRemoved,title_swRemoved_tfidf,title_swRemoved_top_tokens,title_encoding,blocking_keys
0,www.softwarecity.ca//737,Lenovo,Intel. i5-3320M,i5-3320M,Dual-core ( 2 Core ). Core i5,2.60 GHz,None,DDR3 SDRAM. DDR3-1600/PC3-12800. DDR3 SDRAM,DDR3-1600/PC3-12800,320 GB,None,1.80 kg,None,"""Lenovo Thinkpad X230 34352jf Tablet Pc - 12.5...","[lenovo, thinkpad, x230, 34352jf, tablet, pc, ...","(0.021628188358580002, 0.008060458934270659, 0...","[34352jf, french, canada, softwarecity, ca]","[0.01956569217145443, -0.04757802188396454, -0...","[34352jf, french, canada, softwarecity, ca]"
1,www.isupplyhub.com//1256,Acer,1.6 GHz Intel Core i5-4200U. Intel Core I5,None,1.6 GHz Intel Core i5-4200U,1.6 GHz Intel Core i5-4200U,8 GB DDR3L SDRAM,DDR3 SDRAM. 8 GB DDR3L SDRAM,None,500 GB mechanical_hard_drive,None,4.8 pounds,15.02 x 10.08 x 0.90 inches,Amazon.com : Acer Aspire V7-582PG-6479 15.6-In...,"[amazon, com, acer, aspire, v7, 582pg, 6479, 1...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[6479, 582pg, v7, steel, ultrabook]","[-0.041087813675403595, -0.05034055933356285, ...","[6479, 582pg, v7, steel, ultrabook]"
2,www.isupplyhub.com//326,Acer,1.6 GHz Intel Core i5. Intel Core I5,None,1.6 GHz Intel Core i5,1.6 GHz Intel Core i5,4 GB DDR3-SDRAM,DDR3 SDRAM. 4 GB DDR3-SDRAM,None,500 GB mechanical_hard_drive,None,5.2 pounds,15.02 x 10.08 x 1 inches,Amazon.com : Acer Aspire E1-572-6870 15.6 Inch...,"[amazon, com, acer, aspire, e1, 572, 6870, 15,...","(0.0, 0.013086810537060685, 0.0, 0.0, 0.0, 0.0...","[6870, clarinet, 6ghz, laptop, 4200u]","[-0.03485466539859772, -0.057676222175359726, ...","[6870, clarinet, 6ghz, laptop, 4200u]"
3,www.isupplyhub.com//821,HP,None,None,None,None,4 GB SDRAM DDR3,DDR3 SDRAM. 4 GB SDRAM DDR3,None,500 GB,None,4.8 pounds,15.18 x 0.89 x 10.16 inches,"""Amazon.com : 15.6"""" HP 15-f009wm Amd Dual-Cor...","[amazon, com, 15, 6, hp, 15, f009wm, amd, dual...","(0.0, 0.012601489025411704, 0.0, 0.0, 0.0, 0.0...","[dual, certified, refurbished, webcam, 2100]","[-0.016774244606494904, -0.06671489775180817, ...","[dual, certified, refurbished, webcam, 2100]"
4,www.isupplyhub.com//157,Asus,1.7 GHz Core i5-3317U. Intel,None,1.7 GHz Core i5-3317U,1.7 GHz Core i5-3317U,4 GB DDR3,DDR3 SDRAM. 4 GB DDR3,None,256 MB,None,2.9 pounds,8.80 x 0.70 x 12.80 inches,Amazon.com : ASUS UX31A-XB52 13.3-Inch Ultrabo...,"[amazon, com, asus, ux31a, xb52, 13, 3, inch, ...","(0.0, 0.011440724195460843, 0.0, 0.0, 0.0, 0.0...","[3317u, 256gb, xb52, aluminum, asus]","[0.00795380026102066, -0.05642663687467575, -0...","[3317u, 256gb, xb52, aluminum, asus]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,www.vology.com//873,Lenovo ThinkPad X230 2320 - 12.5 '' - Core i5 ...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,180 GB SSD. 180 GB SSD. Lenovo ThinkPad X230 2...,180 GB SSD. 180 GB SSD,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,"""Lenovo ThinkPad X230 2320 - 12.5"""" - Core i5 ...","[lenovo, thinkpad, x230, 2320, 12, 5, core, i5...","(0.12268608279935049, 0.04572302200329186, 0.0...","[2320, 180, 3320m, ssd, x230]","[0.026927268132567406, -0.05049174651503563, -...","[2320, 180, 3320m, ssd, x230]"
339,www.vology.com//823,Lenovo ThinkPad X230 2325 - 12.5 '' - Core i5 ...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,4 GB DDR3 Slots Qty 2 Max RAM Supported 16 GB ...,4 GB DDR3 Slots Qty 2 Max RAM Su